# March Madness Bracket Simulator
By: Jackson Isidor and Alex Sullivan

Making an optimized March Madness Bracket that maximizes total points in standard bracket scoring.

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")

**The Data:**
- Each row represents a matchup between 2 teams
- Identifying info on each time, including year, name, seed, current round, and the round they made it to
- Team statistics from KenPom and Barttovik

In [9]:
matchups = pd.read_csv("https://raw.githubusercontent.com/jacksonisidor/March-Madness-Predictor/main/MM%20Data/matchup_stats.csv")

In [10]:
matchups.head()

,Unnamed: 0,year,team_1,seed_1,round_1,current_round,team_2,seed_2,round_2,badj_em_1,...,win_percent_2,winner,badj_em_diff,wab_diff,barthag_diff,talent_diff,elite_sos_diff,win_percent_diff,pppo_diff,k_off_diff
0,0,2023,Alabama,1,16,64,Texas A&M Corpus Chris,16,64,27.1,...,65.517241,1,28.9,16.2,0.505,62.286,24.154,19.776876,0.008,0.998
1,1,2023,Maryland,8,32,64,West Virginia,9,64,16.5,...,57.575758,1,-3.2,-0.5,-0.032,3.690,-6.315,6.060606,-0.002,0.056
2,2,2023,San Diego St.,5,2,64,College of Charleston,12,64,21.2,...,90.909091,1,9.7,4.1,0.137,37.036,16.740,-9.659091,-0.062,-5.790
3,3,2023,Virginia,4,64,64,Furman,13,32,16.9,...,77.419355,0,9.1,6.1,0.174,50.410,14.426,0.705645,-0.072,-7.131
4,4,2023,Creighton,6,8,64,North Carolina St.,11,64,21.0,...,69.696970,1,5.8,0.8,0.073,12.798,8.184,-6.060606,-0.020,-1.706


# Bracket Scoring

**Function to score bracket:**
- Take in the predicted bracket and actual bracket 
- Use standard scoring used by most march madness bracket sites (cbs, yahoo, etc):
    - 1 point per R64 game
    - 2 points per R32 game
    - 4 points per R16 (sweet sixteen) game
    - 8 points per R8 (elite eight) game
    - 16 points per R4 (final four) game
    - 32 points per R2 (final) game

In [14]:
def score_bracket(predicted, actual):
    
    score = 0
    for (pred_index, pred_matchup), (act_index, act_matchup) in zip(predicted.iterrows(), actual.iterrows()):
        
        if (pred_matchup["team_1"] == act_matchup["team_1"]) and (pred_matchup["prediction"] == act_matchup["winner"] == 1):
            score += 64 / pred_matchup["current_round"]
            
        elif (pred_matchup["team_2"] == act_matchup["team_2"]) and (pred_matchup["prediction"] == act_matchup["winner"] == 0): 
            score += 64 / pred_matchup["current_round"]
            
    return score

# Bracket Simulating
This will involve multiple functions. 
- One main one that goes through the full bracket, calling the other functions
- A function to get the winner information from each round
- A function to create the matchups for each subsequent round

**Winner info function**:

This function gets the information from the winning team in each round and returns a dataframe.

*Notes:* 
- If winner == 1, that means team_1 wins, so I will need only that team's info, same for team 2
- This returns a dataframe consisting of 1 team per row. The 1st row plays the 2nd row, and so on
- I will have to do the same processing I did in the first round where I combined the rows into matchups. Therefore, team_1 and team_2 may switch from round to round for each team.
    - To deal with this, I strip off the _1 or _2 from the stats after getting their info, because the may be renamed in next round depending on placement

In [15]:
def get_winner_info(matchups):
    next_round_teams_list = []
    
    for index, matchup in matchups.iterrows():
        # if team_1 wins, get all info that ends in "_1"
        if matchup["prediction"] == 1:
            winning_team_info = matchup.filter(regex='_1$').rename(lambda x: x[:-2], axis=0)
        # if team_2 wins, get all info that ends in "_2"
        else:
            winning_team_info = matchup.filter(regex='_2$').rename(lambda x: x[:-2], axis=0)
        
        winning_team_info["year"] = matchup["year"]
        winning_team_info["current_round"] = matchup["current_round"] / 2
        
        next_round_teams_list.append(pd.DataFrame(winning_team_info).T)
    
    next_round_teams = pd.concat(next_round_teams_list, ignore_index=True)
        
    return next_round_teams

**New Round Matchups Function:**

This combines pairs of rows into matchups. It will be exactly the same as I did in the processing notebook, but put into a function to be used in each round

In [16]:
def make_matchups(winning_teams):
    matchups = pd.DataFrame(columns=['year', 'team_1', 'seed_1', 'round_1', 'current_round', 'team_2', 'seed_2', 'round_2'])

    matchup_info_list = []
    # iterate through data frame and jump 2 each iteration
    for i in range(0, len(winning_teams)-1, 2):
        team1_info = winning_teams.iloc[i]
        team2_info = winning_teams.iloc[i+1]

        matchup_info = {
            'year': team1_info['year'],
            'team_1': team1_info['team'],
            'seed_1': team1_info['seed'],
            'round_1': team1_info['round'],
            'current_round': team1_info['current_round'],
            'team_2': team2_info['team'],
            'seed_2': team2_info['seed'],
            'round_2': team2_info['round'],
            'badj_em_1': team1_info['badj_em'],
            'wab_1': team1_info['wab'],  
            'barthag_1': team1_info['barthag'],
            'talent_1': team1_info['talent'],
            'elite_sos_1': team1_info['elite_sos'],
            'win_percent_1': team1_info['win_percent'],
            'pppo_1': team1_info['pppo'],
            'k_off_1': team1_info['k_off'],
            'badj_em_2': team2_info['badj_em'],
            'wab_2': team2_info['wab'],  
            'barthag_2': team2_info['barthag'],
            'talent_2': team2_info['talent'],
            'elite_sos_2': team2_info['elite_sos'],
            'win_percent_2': team2_info['win_percent'],
            'pppo_2': team2_info['pppo'],
            'k_off_2': team2_info['k_off']
        }
        matchup_info_list.append(matchup_info)

    matchups = pd.concat([matchups, pd.DataFrame(matchup_info_list)])
        
    # get the stat differences same as before
    stat_variables = ['badj_em', 'wab', 'barthag', 'talent', 'elite_sos', 'win_percent', 'pppo', 'k_off']
    for variable in stat_variables:
        matchups[f'{variable}_diff'] = matchups[f'{variable}_1'] - matchups[f'{variable}_2']
        
    return matchups

**Main Bracket Simulation:**

Pass in the round of 64 for a given year and predict each game, simulating through the rounds using recursion.

*Structure of data*: the winner of the first row plays the winner of the second row, 3rd plays the 4th, etc

In [17]:
def sim_bracket(round_matchups, model):

    # get predictions for each game in the current round and add that column to the df
    preds = model.predict(round_matchups[predictors])
    round_matchups.loc[:, "prediction"] = preds
    
    # add in probabilities too in case I want to identify the most likely upsets
    probs = model.predict_proba(round_matchups[predictors])
    round_matchups.loc[:, "win probability"] = probs[:, 1]

    
    # base case for recursion (we are in the championship round)
    if round_matchups["current_round"].iloc[0] == 2:
        return round_matchups
    
    # pass teams on to the next round in a new df and combine them into new matchups
    next_round_teams = get_winner_info(round_matchups)
    next_round_matchups = make_matchups(next_round_teams)

    # recurse through making a simulated df that mimics the structure of the actual df
    return pd.concat([round_matchups, sim_bracket(next_round_matchups, model)], ignore_index=True)

## Test The Simulation On Various Models
Train a model determined in another notebook with all years except for one. Loop through all the years to perform a manual cross-validation.

**SVM Model (best so far):**
- Parameters: C = 100, gamma = 0.0001, kernel = rbf
- Features: badj_em_diff, barthag_diff, elite_sos_diff, win_percent_diff, pppo_diff

In [18]:
predictors = ["badj_em_diff", "barthag_diff", "elite_sos_diff", "win_percent_diff", "pppo_diff", "wab_diff"]
target = "winner"

# loop through all the years in the data
scores = pd.DataFrame(columns=["year", "score"])
for year in matchups["year"].unique():
    
    # split data leaving out 1 year for testing
    train = matchups[matchups["year"] != year] 
    test = matchups[matchups["year"] == year]
    X_train = train[predictors]
    y_train = train[target]
    
    # train model on rest of the years
    svm_pipeline = make_pipeline(StandardScaler(), 
                         SVC(C=0.1,
                             gamma=0.1,
                             kernel='rbf',
                             probability=True
                         ))

    svm_pipeline.fit(X_train, y_train)
    
    # simulate the test bracket 
    test_r64 = test[test["current_round"] == 64]
    prediction_bracket = sim_bracket(test_r64, svm_pipeline)
    
    # score the test bracket
    score = score_bracket(prediction_bracket, test)
    
    # add to df
    test_bracket_info = pd.DataFrame({'year': [year], 'score': [score]})
    scores = pd.concat([scores, test_bracket_info], ignore_index=True)

In [19]:
scores

,year,score
0,2023,47.0
1,2022,79.0
2,2019,127.0
3,2018,110.0
4,2017,77.0
5,2016,82.0
6,2015,108.0
7,2014,66.0
8,2013,82.0
9,2012,128.0


In [20]:
scores.score.mean()

89.35714285714286

**XGB Model**
- Parameters: colsample_bytree = 0.9, gamma = 5, learning_rate = 0.1, max_depth = 7, n_estimators = 300, subsample = 0.9
- Features: 'badj_em_diff', 'wab_diff', 'barthag_diff', 'talent_diff', 'elite_sos_diff', 'win_percent_diff', 'pppo_diff', 'k_off_diff'

In [21]:
predictors = ['badj_em_diff', 'wab_diff', 'barthag_diff', 'talent_diff', 'elite_sos_diff', 'win_percent_diff', 'pppo_diff', 'k_off_diff']
target = "winner"

# loop through all the years in the data
scores = pd.DataFrame(columns=["year", "score"])
for year in matchups["year"].unique():
    
    # split data leaving out 1 year for testing
    train = matchups[matchups["year"] != year] 
    test = matchups[matchups["year"] == year]
    X_train = train[predictors]
    y_train = train[target]
    
    xgb_pipeline = make_pipeline(StandardScaler(), 
                         XGBClassifier(colsample_bytree=0.9,
                                       gamma=5,
                                       learning_rate=0.01,
                                       max_depth=7,
                                       n_estimators=300,
                                       subsample=0.9
                         ))

    xgb_pipeline.fit(X_train, y_train)
    
    # simulate the test bracket 
    test_r64 = test[test["current_round"] == 64]
    prediction_bracket = sim_bracket(test_r64, xgb_pipeline)
    
    # score the test bracket
    score = score_bracket(prediction_bracket, test)
    
    # add to df
    test_bracket_info = pd.DataFrame({'year': [year], 'score': [score]})
    scores = pd.concat([scores, test_bracket_info], ignore_index=True)

In [22]:
scores

,year,score
0,2023,48.0
1,2022,112.0
2,2019,126.0
3,2018,61.0
4,2017,72.0
5,2016,75.0
6,2015,93.0
7,2014,70.0
8,2013,66.0
9,2012,125.0


In [23]:
scores.score.mean()

90.57142857142857

## Optimizing Simulation
Now, I will try to tune the hyperparameters to optimize the full bracket simulation, rather than individual matchups. This may yield different results.

Using the xgboost model:

*Note:* Since I am running a full bracket simulation for each set of parameters, I have to do a manual grid search (a bunch of for loops). Depending on how computationaly expensive this is, I may reduce the grid.

In [91]:
predictors = ['badj_em_diff', 'wab_diff', 'barthag_diff', 'talent_diff', 'elite_sos_diff', 'win_percent_diff', 'pppo_diff', 'k_off_diff']
target = "winner"

param_scores = {}
for estimator in [100, 200, 300]:
    for depth in [3, 5, 7]:
        for lr in [0.01, 0.1, 0.2]:
            for ss in [0.8, 0.9, 1.0]:
                for csbt in [0.8, 0.9, 1.0]:
                    for g in [0, 1, 5]:
                        
                        # Loop through all the years in the data with the current set of params
                        scores = []
                        for year in matchups["year"].unique():
                            # Split data leaving out 1 year for testing
                            train = matchups[matchups["year"] != year] 
                            test = matchups[matchups["year"] == year]
                            X_train = train[predictors]
                            y_train = train[target]

                            # Train model on rest of the years
                            xgb_pipeline = make_pipeline(StandardScaler(), 
                                                 XGBClassifier(n_estimators=estimator,
                                                               max_depth=depth,
                                                               learning_rate=lr,
                                                               subsample=ss,
                                                               colsample_bytree=csbt,
                                                               gamma=g
                                                 ))

                            xgb_pipeline.fit(X_train, y_train)

                            # Simulate the test bracket 
                            test_r64 = test[test["current_round"] == 64]
                            prediction_bracket = sim_bracket(test_r64, xgb_pipeline)

                            # Score the test bracket
                            score = score_bracket(prediction_bracket, test)
                            scores.append(score)
                        
                        # Get the average score for this set of parameters and add it to the dictionary
                        cv_score = np.mean(scores)
                        param_scores[(estimator, depth, lr, ss, csbt, g)] = cv_score

In [92]:
df = pd.DataFrame(list(param_scores.items()), columns=['Parameters', 'Score'])
df.sort_values(by="Score", ascending=False).head()

,Parameters,Score
648,"(300, 7, 0.01, 0.8, 0.8, 0)",97.785714
421,"(200, 7, 0.01, 0.9, 1.0, 1)",97.571429
582,"(300, 5, 0.01, 0.9, 1.0, 0)",94.785714
551,"(300, 3, 0.2, 0.9, 0.8, 5)",94.642857
500,"(300, 3, 0.01, 0.9, 0.9, 5)",94.428571


**Test the new best parameters, same as before:**

In [24]:
predictors = ['badj_em_diff', 'wab_diff', 'barthag_diff', 'talent_diff', 'elite_sos_diff', 'win_percent_diff', 'pppo_diff', 'k_off_diff']
target = "winner"

scores = pd.DataFrame(columns=["year", "score"])
for year in matchups["year"].unique():
    # Split data leaving out 1 year for testing
    train = matchups[matchups["year"] != year] 
    test = matchups[matchups["year"] == year]
    X_train = train[predictors]
    y_train = train[target]

    # Train model on rest of the years
    xgb_pipeline = make_pipeline(StandardScaler(), 
                                  XGBClassifier(n_estimators=300,
                                                max_depth=7,
                                                learning_rate=0.01,
                                                subsample=0.8,
                                                colsample_bytree=0.8,
                                                gamma=0
                                                 ))

    xgb_pipeline.fit(X_train, y_train)

    # Simulate the test bracket 
    test_r64 = test[test["current_round"] == 64]
    prediction_bracket = sim_bracket(test_r64, xgb_pipeline)

    # Score the test bracket
    score = score_bracket(prediction_bracket, test)
    
    # add to df
    test_bracket_info = pd.DataFrame({'year': [year], 'score': [score]})
    scores = pd.concat([scores, test_bracket_info], ignore_index=True)

In [25]:
scores

,year,score
0,2023,46.0
1,2022,110.0
2,2019,69.0
3,2018,63.0
4,2017,116.0
5,2016,122.0
6,2015,94.0
7,2014,64.0
8,2013,132.0
9,2012,137.0


In [26]:
scores.score.mean()

97.78571428571429

In [27]:
# just want to check the average without 2023 because 2023 was extra random in my opinion
scores[scores["year"] != 2023].score.mean()

101.76923076923077

Running the grid search around the simulation rather than for each matchup improved the overall score, as expected. This is due to some strategy in picking a full bracket because later rounds are worth higher point values, so the model discovered that getting the champion or a few final four teams was really valuable. 

Historical bracket info is mostly hidden by ESPN, CBS, etc, but the NCAA does include this graphic on their website: 

<img src="bracket_averages.png" alt="NCAA Bracket Averages" width="800" height="400"/>

First, this shows that picking just the higher seed every time does significantly better than the average person. The average user score over this time frame is 67, while the average seed-based bracket score is 88.

Our bracket prediction model has an average score of 94 over this time frame and 98 over all the available years, consistently performing better than both.